<a href="https://colab.research.google.com/github/aiaidada/MHealth/blob/main/dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, upload your files on Colab, and then Unzip it using the code below



In [100]:
!unzip dataset.zip

Creats a dataset from the data you have and creates a CNN model

In [105]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
import torch.nn.functional as F

# Define a custom dataset class
class CustomDataset(Dataset):
    def __init__(self, file_paths, labels , transform):
        self.file_paths = file_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        img = Image.open(self.file_paths[idx])
        label = self.labels[idx]

        if self.transform:
            img = self.transform(img)

        return img, label

# Define the paths to your raw image files and labels

church_labels = [0] * 30   # Assign label 0 to 'Church' class
face_labels = [1] * 30 # Assign label 1 to 'face' class

# Combine the file paths and labels
labels = church_labels + face_labels


# Define the transforms
transform = transforms.Compose([
    transforms.ToTensor(),         # Convert to tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5) )  # Normalize
])

# Create an instance of custom dataset
custom_dataset = CustomDataset(myFileList, labels, transform=transform)

# Split into training and validation sets
train_size = int(0.8 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(custom_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1)



class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(59536, 2500)
        self.fc2 = nn.Linear(2500, 84)
        self.fc3 = nn.Linear(84, 2)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()


In [106]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

Trains the model

In [110]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs.unsqueeze(0), labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0

print('Finished Training')

Finished Training


Shows prediction of the model

In [132]:
from PIL import Image
for i in range (1 ,31):
  im = Image.open(f'/content/dataset/Church/Church ({i}).png')

  im = transform(im)
  print(net(im))

tensor([-6.8810,  6.7496], grad_fn=<ViewBackward0>)
tensor([-6.7161,  6.7234], grad_fn=<ViewBackward0>)
tensor([-6.8484,  6.6560], grad_fn=<ViewBackward0>)
tensor([-4.0421,  4.4033], grad_fn=<ViewBackward0>)
tensor([-4.7944,  5.1884], grad_fn=<ViewBackward0>)
tensor([-7.4592,  7.3596], grad_fn=<ViewBackward0>)
tensor([-4.1011,  4.1658], grad_fn=<ViewBackward0>)
tensor([-6.8117,  6.7876], grad_fn=<ViewBackward0>)
tensor([-3.9915,  3.9569], grad_fn=<ViewBackward0>)
tensor([-5.1175,  5.3928], grad_fn=<ViewBackward0>)
tensor([-7.7193,  7.6380], grad_fn=<ViewBackward0>)
tensor([-5.6344,  5.9492], grad_fn=<ViewBackward0>)
tensor([-3.6651,  3.8059], grad_fn=<ViewBackward0>)
tensor([-4.9814,  5.0839], grad_fn=<ViewBackward0>)
tensor([-4.7314,  4.6347], grad_fn=<ViewBackward0>)
tensor([-6.3094,  6.3053], grad_fn=<ViewBackward0>)
tensor([-2.9181,  3.6512], grad_fn=<ViewBackward0>)
tensor([-5.1554,  5.1788], grad_fn=<ViewBackward0>)
tensor([-3.9065,  4.7732], grad_fn=<ViewBackward0>)
tensor([-5.9

tensor([-7.4592,  7.3596], grad_fn=<ViewBackward0>)


In [111]:
print(net(img_tensor))

tensor([ 3.8814, -3.9475], grad_fn=<ViewBackward0>)


In [81]:
print(model(img_tensor))

tensor([[ 15.6228, -41.6523],
        [  5.0240, -13.6501],
        [  3.7017,  -9.5308],
        [  3.6399,  -9.5020]], grad_fn=<AddmmBackward0>)


In [32]:
from PIL import Image
import numpy as np
import sys
import os
import csv
# default format can be changed as needed
def createFileList(myDir, format='.png'):
    fileList = []
    print(myDir)
    labels = ['Church' , 'face']
    keywords = {"K" : "1","U": "0",} # keys and values to be changed as needed
    for root, dirs, files in os.walk(myDir, topdown=True):
      for name in files:
            if name.endswith(format):
                fullName = os.path.join(root, name)
                fileList.append(fullName)
    return fileList
# load the original image
myFileList  = createFileList('/content/dataset')


for file in myFileList:
    img_file = Image.open(file)
    width, height = img_file.size
    format = img_file.format
    mode = img_file.mode

# Save Greyscale values
    value = np.asarray(img_file.getdata(), dtype=np.int).reshape((3 , width, height))
    value = value.flatten()
    m = file.split('/')
    if file.find('face'):
      i = 0
    else:
      i = 1
    value = np.append(value, m[3])

    with open("church_face.csv", 'a') as f:
      writer = csv.writer(f)
      writer.writerow(value)

/content/dataset


<ipython-input-32-5536ed9a64dd>:29: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  value = np.asarray(img_file.getdata(), dtype=np.int).reshape((3 , width, height))
